In [3]:
!pip install numpy tensorflow
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import warnings
warnings.filterwarnings('ignore')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 28.5 MB/s  0:00:00m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 20.8 MB/s  0:00:17m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 38.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 34.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 36.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 37.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 35.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 41.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 36.6 MB/s  0:00:00m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 39.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27/27 [tensorflow]7 [tensorflow]-py]a-server]


ModuleNotFoundError: No module named 'pandas'

In [1]:
# TensorFlow and Keras imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, regularizers, callbacks
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
data_dir = "data"
img_size = (224, 224)
img_channels = 3
batch_size = 32

In [ ]:
train = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    image_size=img_size,
    batch_size=batch_size,
    color_mode='rgb',
    shuffle=True,
    validation_split=0.2,
    subset='training',
    seed=50)
test = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    image_size=img_size,
    batch_size=batch_size,
    color_mode='rgb',
    shuffle=True,
    validation_split=0.2,
    subset='validation',
    seed=50)


In [ ]:
train.class_names

In [ ]:
normalization_layer = layers.Rescaling(1./255)

In [ ]:
augmentation_layer = keras.Sequential([
    layers.RandomRotation(factor=30/360,fill_mode='constant',fill_value=0.0),
    layers.RandomZoom(height_factor=0.2,width_factor=0.2,fill_mode='constant',fill_value=0.0),
    layers.RandomBrightness(factor=[0.8,1.1]),
    layers.RandomContrast(factor=0.2),
    layers.RandomTranslation(height_factor=0.1,width_factor=0.1,fill_mode='constant',fill_value=0.0)
], name="augmentation_layer")


In [ ]:
def preprocess_data(data,label):
    data = normalization_layer(data)
    data = augmentation_layer(data, training=True)
    return data,label

In [ ]:
train_x = train.map(preprocess_data)
test_x = test.map(preprocess_data)

In [4]:
base_model = MobileNetV2(
            input_shape=img_size + (img_channels,),
            include_top=False,           # Remove ImageNet classification head
            weights='imagenet',          # Pre-trained on ImageNet
            pooling='avg'                # Global average pooling output
        )

base_model.trainable = False
trainable_count = sum([layer.trainable for layer in base_model.layers])
print(f"   Frozen layers: {len(base_model.layers) - trainable_count}")
print(f"   Trainable layers: {trainable_count}")


NameError: name 'MobileNetV2' is not defined

In [5]:
from tensorflow.keras import regularizers

2026-01-08 02:54:39.828849: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-08 02:54:40.712458: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-08 02:54:43.909349: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
/usr/local/python/3.12.1/lib/python3.12/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [6]:
input = layers.Input(shape=img_size + (img_channels,),name="input_layer")
x = augmentation_layer(input)
x = layers.Rescaling(1./255)(x)
x = base_model(x, training=False)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.0001))(x)
x = layers.BatchNormalization(name='batch_norm')(x)
x = layers.Dropout(0.2, name='dropout_final')(x)
output = layers.Dense(len(train.class_names), activation='softmax', name='output_layer')(x)

NameError: name 'layers' is not defined

In [7]:
model = keras.Model(inputs=input, outputs=ouput)
model.summary()

NameError: name 'keras' is not defined

In [8]:
optimizer = keras.optimizers.Adam(
            learning_rate=0.001,
            beta_1=0.9,
            beta_2=0.999,
            epsilon=1e-07
        )
loss = keras.losses.SparseCategoricalCrossentropy()
metrics = [
            'accuracy',
            keras.metrics.Precision(name='precision'),
            keras.metrics.Recall(name='recall'),
            keras.metrics.AUC(name='auc')  # Area Under ROC Curve
        ]

NameError: name 'keras' is not defined

In [9]:
model.compile(
            optimizer=optimizer,
            loss=loss,
            metrics=metrics
        )

NameError: name 'model' is not defined

In [ ]:
hist = model.fit(train_x, epochs=10, validation_data=test_x)